In [218]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [219]:
data = pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
data.head()

In [220]:
data.corr()

In [221]:
data= data.drop('CustomerID',axis=1)

In [222]:
from sklearn.preprocessing import LabelEncoder
data['Gender'] = LabelEncoder().fit_transform(data['Gender'])

In [223]:
data.corr()

In [224]:
import matplotlib.pyplot as plt
import seaborn as sns

# visualization Normal data

In [225]:
for i in data.columns:
    
    sns.distplot(data[i])
    plt.show()

In [226]:
from sklearn.preprocessing import StandardScaler
data.columns

In [227]:
for i in ['Age','Annual Income (k$)','Spending Score (1-100)']:
    data[i] = StandardScaler().fit_transform(data[[i]])

# Visualization Normalize Data

In [228]:
for i in enumerate(['Age','Annual Income (k$)','Spending Score (1-100)']):
    plt.subplot(2,2,i[0]+1)
    sns.distplot(data[i[1]])

In [229]:
from sklearn.cluster import KMeans

In [230]:
data.columns

# Using 2 Features

In [231]:
data = data[['Annual Income (k$)','Spending Score (1-100)']]

# Fit model using 3 clusters

In [232]:
model = KMeans(n_clusters=3)
model.fit(data)

In [233]:
model.inertia_,model.n_iter_

In [234]:
model.labels_

> 1. * # Elbow Method

In [235]:
elbow = []
for i in range(2,10):
    model = KMeans(n_clusters = i ,init='k-means++', n_init = 10 ,max_iter=100, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan')
    model.fit(data)
    elbow.append(model.inertia_)

In [236]:
plt.plot(elbow)

In [237]:
from sklearn.metrics import silhouette_score


In [238]:
sil_score =[]
for i in range(2,10):
    model = KMeans(n_clusters = i ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 42  , algorithm='elkan')
    model.fit(data)
    score = silhouette_score(data,model.labels_)
    sil_score.append(score)
    

In [239]:
plt.plot(sil_score)

# Got the best result using silhouette score and elbow score . cluster =3

In [240]:
model = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 42 , algorithm='elkan'))
model.fit(data)
model.inertia_,silhouette_score(data,model.labels_)

# Let's try PCA (Principle Component Analysis)

In [241]:
from sklearn.decomposition import PCA

In [242]:
pca =PCA(n_components=2)
pcadata = pca.fit_transform(data)

In [243]:
model = (KMeans(n_clusters = 3 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 42 , algorithm='elkan'))
model.fit(pcadata)
model.inertia_,silhouette_score(pcadata,model.labels_)

In [244]:
from sklearn.pipeline import Pipeline

In [245]:
preprocessor = Pipeline(
    [
        ("scaler", StandardScaler()),
         ("pca", PCA(n_components=2, random_state=42)),
        
    ]
    )

In [246]:
cluster = Pipeline(
[
    ('kmeans',KMeans(n_clusters = 3 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 42 , algorithm='elkan'))
]
)

In [247]:
pipe = Pipeline([
    ('preprocessor',preprocessor),
    ('cluster',cluster)
])

In [248]:
pipe.fit(data)

In [249]:
label = pipe['cluster']['kmeans'].labels_
label

In [250]:
silhouette_score(data,label)

In [251]:
data.columns


# visualization Clustered Data 


In [272]:
import seaborn as sns
import matplotlib.pyplot as plt
c=['red','green','blue']
sns.scatterplot(x=data['Annual Income (k$)'],y=data['Spending Score (1-100)'],palette='Accent_r',hue = label,s=50)


# Trying Feature engineering and analysis more

In [253]:
data = pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
data.head()

# Now New feature is "Savings"

In [254]:
data['savings'] = data['Annual Income (k$)'] - data['Spending Score (1-100)']
data.head()

# Seems Savings is now very important feature

In [255]:
data.corr()

In [256]:
data = data.drop(['CustomerID'],axis=1)

# just 3 features

In [257]:
data = data[['Annual Income (k$)','Spending Score (1-100)','savings']]

In [258]:
for i in data.columns:
    data[i] = StandardScaler().fit_transform(data[[i]])

In [259]:
elbow=[]
sscore = []
for i in range(2,11):
    model = KMeans(n_clusters=i,init='k-means++', n_init = 10 ,max_iter=300,  random_state= 42 , algorithm='elkan')
    model.fit(data)
    elbow.append(model.inertia_)
    score = silhouette_score(data,model.labels_)
    sscore.append(score)

In [260]:
plt.plot(elbow)

In [261]:
plt.plot(sscore)

In [262]:
model = KMeans(n_clusters = 4)
model.fit(data)
silhouette_score(data,model.labels_)

In [263]:
pca.fit(data)

In [264]:
preprocessor = Pipeline(
    [
        ("scaler", StandardScaler()),
         ("pca", PCA(n_components=2, random_state=42)),
        
    ]
    )

In [265]:
cluster = Pipeline(
[
    ('kmean',KMeans(n_clusters=4,init='k-means++', n_init = 10 ,max_iter=300,  random_state= 42 , algorithm='elkan'))
]
)

In [266]:
pca = Pipeline(
[
    ('preprocessor',preprocessor),
    ('cluster',cluster)
])

In [267]:
pca.fit(data)

In [268]:
pcamodel = PCA(n_components=2, random_state=42)
pcadata = pcamodel.fit_transform(data)

In [269]:
silhouette_score(pcadata,pca['cluster']['kmean'].labels_)*100

In [270]:
data.columns
label = pca['cluster']['kmean'].labels_

In [271]:
sns.scatterplot(x = data['Annual Income (k$)'],y = data['savings'],palette='Accent_r',hue = label)

# ..................... If this kernel help you, let me know, and support :) :* ......